In [46]:
import csv
import requests
from bs4 import BeautifulSoup

url = "https://www.haman.go.kr/02828/02829.web"
res = requests.get(url)
res.encoding = "utf-8"
soup = BeautifulSoup(res.text, "html.parser")

rows = []
columns = soup.select("div.sitemap1.even-grid.evenmix-1234 > div.column")

for col in columns:
    main_title = col.select_one("a.d2").get_text(strip=True)
    for mid_li in col.select("ul.bu.mgt0 > li"):
        mid_title = mid_li.find("a", recursive=False).get_text(strip=True)
        for sub_li in mid_li.select("ul.bu.mgt0 > li"):
            a = sub_li.find("a", recursive=False)
            sub_title = a.get_text(strip=True)
            href = a["href"]
            link = href if href.startswith("http") else "https://www.haman.go.kr" + href
            rows.append([main_title, mid_title, sub_title, link])

with open("sitemap_links.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["대주제", "중주제", "소주제", "링크"])
    writer.writerows(rows)

print(f"{len(rows)}개 항목을 sitemap_links.csv에 저장했습니다.")


193개 항목을 sitemap_links.csv에 저장했습니다.


In [62]:
import csv
import requests
from bs4 import BeautifulSoup

INPUT_CSV  = "sitemap_links.csv"
OUTPUT_CSV = "sitemap_full_contents.csv"

# 제외할 안내문 키워드 목록
FILTER_KEYWORDS = [
    "함안으로 떠나는 여행맞춤정보",
    "관광객의 취향을 반영한 최적의 관광지를 추천해드립니다",
    "Q1", "Q2", "Q3",
    "안내도 바로보기"
]

def is_boilerplate(text):
    return any(text.startswith(kw) for kw in FILTER_KEYWORDS)

with open(INPUT_CSV, newline="", encoding="utf-8") as fin, \
     open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as fout:

    reader = csv.DictReader(fin)
    writer = csv.DictWriter(fout, fieldnames=["대주제","중주제","소주제","링크","상세정보"])
    writer.writeheader()

    for row in reader:
        main, mid, sub, url = row["대주제"], row["중주제"], row["소주제"], row["링크"]
        detail = ""
        try:
            res = requests.get(url, timeout=10)
            res.encoding = "utf-8"
            soup = BeautifulSoup(res.text, "html.parser")

            # wrap1 우선
            wrap = soup.select_one("div.info3cont > div.wrap1") or \
                   soup.select_one("div.tour1info3.more.full") or \
                   soup.select_one("div.infobox2") or \
                   soup.select_one("div.tg2")
            if wrap:
                # 텍스트 전체 추출, 라인별로 분리 후 불필요 항목 제외
                texts = [line.strip() for line in wrap.get_text(separator="\n").split("\n") if line.strip()]
                filtered = [t for t in texts if not is_boilerplate(t)]
                detail = "\n".join(filtered)
        except Exception:
            detail = ""

        writer.writerow({
            "대주제": main,
            "중주제": mid,
            "소주제": sub,
            "링크": url,
            "상세정보": detail
        })

print(f"완료: {OUTPUT_CSV}에 상세정보를 저장했습니다.")


완료: sitemap_full_contents.csv에 상세정보를 저장했습니다.


In [64]:
import csv
import requests
from bs4 import BeautifulSoup

url = "https://www.haman.go.kr/02373/02374/02379.web"
res = requests.get(url, timeout=10)
res.encoding = "utf-8"
soup = BeautifulSoup(res.text, "html.parser")

rows = []
for li in soup.select("div.tour1list1 ol > li.li1"):
    num = li.select_one("span.num").get_text(strip=True)
    name = li.select_one("strong.h1 a").get_text(strip=True)
    href = li.select_one("strong.h1 a")["href"]
    detail_link = requests.compat.urljoin(url, href)
    map_a = li.select_one("div.btns a.btn-link-path")
    address = map_a["data-address"].strip() if map_a else ""
    rows.append([num, name, detail_link, address])

with open("restaurant_list.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["순번", "음식점명", "상세페이지 링크", "주소"])
    writer.writerows(rows)

print(f"{len(rows)}개 항목을 restaurant_list.csv에 저장했습니다.")


25개 항목을 restaurant_list.csv에 저장했습니다.


In [71]:
import csv
import requests
from bs4 import BeautifulSoup

# 함안멋집(카페) 페이지 URL
URL = "https://www.haman.go.kr/02373/02374/03546.web"
OUTPUT_CSV = "hanan_cafe_list.csv"

res = requests.get(URL, timeout=10)
res.encoding = "utf-8"
soup = BeautifulSoup(res.text, "html.parser")

rows = []
for li in soup.select("div.tour1list1 ol > li.li1"):
    num_tag = li.select_one("span.num")
    num = num_tag.get_text(strip=True) if num_tag else ""
    
    name_tag = li.select_one("strong.h1 a")
    name = name_tag.get_text(strip=True) if name_tag else ""
    href = name_tag["href"] if name_tag and name_tag.has_attr("href") else ""
    detail_link = requests.compat.urljoin(URL, href)
    
    map_a = li.select_one("div.btns a.btn-link-path")
    address = map_a["data-address"].strip() if map_a and map_a.has_attr("data-address") else ""
    lat = map_a["data-y"].strip()     if map_a and map_a.has_attr("data-y")     else ""
    lon = map_a["data-x"].strip()     if map_a and map_a.has_attr("data-x")     else ""
    
    rows.append([num, name, detail_link, address, lat, lon])

with open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["순번", "카페명", "상세페이지 링크", "주소", "위도", "경도"])
    writer.writerows(rows)

print(f"{len(rows)}개 항목을 '{OUTPUT_CSV}'에 저장했습니다.")


13개 항목을 'hanan_cafe_list.csv'에 저장했습니다.


In [70]:
import csv
import requests
from bs4 import BeautifulSoup

URL = "https://www.haman.go.kr/02373/02374/02380.web"
OUTPUT_CSV = "hanan_matjip_detailed.csv"

res = requests.get(URL, timeout=10)
res.encoding = "utf-8"
soup = BeautifulSoup(res.text, "html.parser")

rows = []
for li in soup.select("div.tour1list1 ol > li.li1"):
    num_tag = li.select_one("span.num")
    num = num_tag.get_text(strip=True) if num_tag else ""
    
    name_tag = li.select_one("strong.h1 a")
    name = name_tag.get_text(strip=True) if name_tag else ""
    href = name_tag["href"] if name_tag and name_tag.has_attr("href") else ""
    detail_link = requests.compat.urljoin(URL, href)
    
    map_a = li.select_one("div.btns a.btn-link-path")
    address = map_a["data-address"].strip() if map_a and map_a.has_attr("data-address") else ""
    lat = map_a["data-y"].strip()     if map_a and map_a.has_attr("data-y")     else ""
    lon = map_a["data-x"].strip()     if map_a and map_a.has_attr("data-x")     else ""
    
    rows.append([num, name, detail_link, address, lat, lon])

with open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["순번", "음식점명", "상세페이지 링크", "주소", "위도", "경도"])
    writer.writerows(rows)

print(f"{len(rows)}개 항목을 {OUTPUT_CSV}에 저장했습니다.")


34개 항목을 hanan_matjip_detailed.csv에 저장했습니다.


In [68]:
import csv
import requests
from bs4 import BeautifulSoup

url = "https://www.haman.go.kr/02373/02374/02381.web"
res = requests.get(url, timeout=10)
res.encoding = "utf-8"
soup = BeautifulSoup(res.text, "html.parser")

with open("ansim_sikdang.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["연번", "업종", "업태", "업소명", "소재지", "전화번호"])

    for tr in soup.select("table.t3.tttac.ttvam tbody.tdtac tr"):
        cells = tr.find_all(["th", "td"])
        num      = cells[0].get_text(strip=True)
        type_    = cells[1].get_text(strip=True)
        category = cells[2].get_text(strip=True)
        name     = cells[3].get_text(strip=True)
        address  = cells[4].get_text(strip=True)

        # 전화번호 셀 추출 and 방어
        phone_cell = cells[5]
        span = phone_cell.select_one("span.t1")
        if span:
            phone = span.get_text(strip=True)
        else:
            # <a> 자체에 텍스트가 있으면 그걸 쓰거나, td 전체 텍스트 사용
            phone = phone_cell.get_text(strip=True)

        writer.writerow([num, type_, category, name, address, phone])


print("완료: ansim_sikdang.csv 파일에 저장되었습니다.")


완료: ansim_sikdang.csv 파일에 저장되었습니다.


In [73]:
import csv
import requests
from bs4 import BeautifulSoup

# Legend list pages
pages = [
    "https://www.haman.go.kr/01834/01862/02166.web?cpage=1",
    "https://www.haman.go.kr/01834/01862/02166.web?cpage=2",
    "https://www.haman.go.kr/01834/01862/02166.web?cpage=3"
]

output_file = "legends_links.csv"

with open(output_file, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["제목", "링크"])
    
    for page in pages:
        res = requests.get(page, timeout=10)
        res.encoding = "utf-8"
        soup = BeautifulSoup(res.text, "html.parser")
        
        # 각 전설 항목의 <a> 태그 선택
        for a in soup.select("li > a[href*='?amode=view']"):
            title_tag = a.select_one("strong.h1")
            title = title_tag.get_text(strip=True) if title_tag else ""
            href = a["href"]
            # 절대 URL 생성
            link = href if href.startswith("http") else requests.compat.urljoin(page, href)
            writer.writerow([title, link])

print(f"{output_file}에 저장 완료")


legends_links.csv에 저장 완료


In [76]:
import csv
import requests
from bs4 import BeautifulSoup

INPUT_CSV = "legends_links.csv"
OUTPUT_CSV = "legends_with_details.csv"

with open(INPUT_CSV, newline="", encoding="utf-8") as fin, \
     open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as fout:

    reader = csv.DictReader(fin)
    fieldnames = reader.fieldnames + ["상세정보"]
    writer = csv.DictWriter(fout, fieldnames=fieldnames)
    writer.writeheader()

    for row in reader:
        url = row["링크"]
        detail_texts = []
        try:
            res = requests.get(url, timeout=10)
            res.encoding = "utf-8"
            soup = BeautifulSoup(res.text, "html.parser")
            substance = soup.select_one("div.wrap1 div.substance")
            if substance:
                # 줄바꿈 구분으로 모든 텍스트 추출
                lines = [line.strip() for line in substance.get_text(separator="\n").split("\n") if line.strip()]
                detail_texts = lines
        except Exception:
            detail_texts = []

        row["상세정보"] = "\n".join(detail_texts)
        writer.writerow(row)

print(f"완료: {OUTPUT_CSV}에 상세정보 열을 추가하여 저장했습니다.")


완료: legends_with_details.csv에 상세정보 열을 추가하여 저장했습니다.


In [77]:
import csv
import requests
from bs4 import BeautifulSoup

# 1) 인물 페이지 URL 리스트
base = "https://www.haman.go.kr/01834/01862/"
indices = list(range(2186, 2210))  # 02186 ~ 02209
urls = [f"{base}{idx:05d}.web?amode=view&idx={idx}" for idx in indices]

# 2) CSV 파일 생성
with open("persons.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["이름", "링크", "특칭", "설명"])

    for url in urls:
        res = requests.get(url, timeout=10)
        res.encoding = "utf-8"
        soup = BeautifulSoup(res.text, "html.parser")
        
        # 이름: 파일명에서 추출하거나 <strong.h1> 대신 목차에서 미리 알고 있는 경우 대체
        # 여기서는 특칭에서 괄호 앞까지를 이름으로 취급
        h2 = soup.select_one("h2.h1")
        special = h2.get_text(" ", strip=True) if h2 else ""
        # 예: "홍건적을 물리친, 이방실 장군 (1298~1362)"
        # 이름은 마지막 쉼표 이후 첫 어절로 추출
        name = special.split(",")[-1].split()[0] if "," in special else special.split()[0]
        
        # 설명: wrap1 > substance 내 <p>
        desc_block = soup.select_one("div.wrap1 div.substance")
        if desc_block:
            ps = [p.get_text(" ", strip=True) for p in desc_block.find_all("p") if p.get_text(strip=True)]
            description = "\n".join(ps)
        else:
            description = ""
        
        writer.writerow([name, url, special, description])

print("완료: persons.csv에 인물별 이름·링크·특칭·설명을 저장했습니다.")


완료: persons.csv에 인물별 이름·링크·특칭·설명을 저장했습니다.


In [78]:
import pandas as pd

# 파일 경로
INPUT_CSV = "sitemap_full_contents.csv"
OUTPUT_CSV = "sitemap_full_contents_filtered.csv"

# CSV 불러오기
df = pd.read_csv(INPUT_CSV, encoding="utf-8")

# '상세정보' 열이 비어 있거나 NaN인 행 제거
df_filtered = df[df["상세정보"].notna() & (df["상세정보"].str.strip() != "")]

# 결과를 새 파일로 저장
df_filtered.to_csv(OUTPUT_CSV, index=False, encoding="utf-8")

print(f"완료: '상세정보'가 비어 있지 않은 {len(df_filtered)}개 행을 {OUTPUT_CSV}에 저장했습니다.")


완료: '상세정보'가 비어 있지 않은 130개 행을 sitemap_full_contents_filtered.csv에 저장했습니다.
